# Training Intertidal Classification of Europe: Categorising Reflectance of Emerged Areas of Marine vegetation using Sentinel 2

# ICE CREAMS

Import fastbook module and required elements within it to run a tabular learner.

In [1]:
from fastai.tabular.all import *
import glob
import pandas as pd

Read in training data in wide format.
Each version of the model is stored in a separate folder inside Data/Input/Training of the repository. csv files have also been separated into 100,000 row files to minimise file sizes. Therefore, we need to combine them all to create the full training dataframe. 

Training data for ICE CREAMS has 26 features and one label/y column with 9 classes. Features are 12 Bottom of Atmosphere reflectance columns, 12 Min-Max standardised reflectance columns, and 2 columns for Normalised Difference Vegetation Index (NDVI) and Normalised Difference Water Index (NDWI).

In [2]:
Training_Location_V1_1=glob.glob("Data/Input/Training/V1_1/*.csv")
#Training_Location_V1_2=glob.glob("Data/Input/Training/V1_2/*.csv")
#Training_Location_V1_3=glob.glob("Data/Input/Training/V1_3/*.csv")

In [3]:
Training_df_V1_1 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_1), ignore_index=True)
#Training_df_V1_2 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_2), ignore_index=True)
#Training_df_V1_3 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_3), ignore_index=True)

In [4]:
#df_nn = pd.concat([Training_df_V1_1,Training_df_V1_2,Training_df_V1_3])
df_nn=Training_df_V1_1

In [5]:
df_nn

,True_Class,Reflectance_B02,Reflectance_B03,Reflectance_B04,Reflectance_B08,Reflectance_B05,Reflectance_B06,Reflectance_B07,Reflectance_B11,Reflectance_B12,...,Reflectance_Stan_B05,Reflectance_Stan_B06,Reflectance_Stan_B07,Reflectance_Stan_B11,Reflectance_Stan_B12,Reflectance_Stan_B8A,Reflectance_Stan_B01,Reflectance_Stan_B09,NDVI,NDWI
0,Microphytobenthos,824,1074,944,1910,1535,1746,1759,1204,661,...,0.699760,0.868695,0.879103,0.434748,0.0,0.919936,0.045637,0.918335,0.338472,-0.280161
1,Microphytobenthos,852,1122,944,1958,1535,1746,1759,1204,661,...,0.673863,0.836546,0.846569,0.418658,0.0,0.885891,0.043948,0.884348,0.349414,-0.271429
2,Microphytobenthos,891,1142,986,1676,1614,1775,1803,1174,652,...,0.832180,0.971453,0.995675,0.451557,0.0,0.987024,0.057093,1.000000,0.259204,-0.189496
3,Microphytobenthos,869,1222,1012,1910,1614,1775,1803,1174,652,...,0.764706,0.892687,0.914944,0.414944,0.0,0.906995,0.052464,0.918919,0.307324,-0.219668
4,Microphytobenthos,754,1064,844,2120,1625,1893,2011,1301,710,...,0.648936,0.839007,0.922695,0.419149,0.0,0.898582,0.023404,0.880851,0.430499,-0.331658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371120,Bare Sediment,758,1050,1104,1398,1326,1278,1375,291,108,...,0.944186,0.906977,0.982171,0.141860,0.0,0.972093,0.325581,0.741085,0.117506,-0.142157
371121,Bare Sediment,779,1052,1112,1362,1326,1278,1375,291,108,...,0.961326,0.923441,1.000000,0.144436,0.0,0.989740,0.331492,0.754538,0.101051,-0.128418
371122,Bare Sediment,747,1028,1112,1342,1318,1273,1349,214,87,...,0.975436,0.939778,1.000000,0.100634,0.0,0.977813,0.332805,0.697306,0.093725,-0.132489
371123,Bare Sediment,756,992,1092,1300,1318,1273,1349,214,87,...,0.975436,0.939778,1.000000,0.100634,0.0,0.977813,0.332805,0.697306,0.086957,-0.134380


Our labelled column is called True_Class. 

In [6]:
dep_var = "True_Class"
splits = RandomSplitter(valid_pct=0.2)(range_of(df_nn))


Select which columns are categoric and which are continuous, ICE CREAMS uses just continuous features.

In [7]:
cont_nn,cat_nn = cont_cat_split(df_nn, dep_var=dep_var)

Minimal pre processing is carried out but if any missing data is present it should be filled. 

In [8]:
procs_nn = [FillMissing]
to_nn = TabularPandas(df_nn, procs_nn, cat_nn, cont_nn,
                      splits=splits, 
                      y_names=dep_var, y_block=CategoryBlock())

In [9]:
dls = to_nn.dataloaders()

In [10]:
y = to_nn.train.y

Cleate the learner, which is a tabular learner with 9 output classes.

In [11]:
learn = tabular_learner(dls,n_out=9)

Fine tune the learner/model over 20 epochs. Default fine tuning is carried out with ADAM optimiser.

In [12]:
learn.fine_tune(20)

epoch,train_loss,valid_loss,time
0,0.153854,0.173743,00:36


epoch,train_loss,valid_loss,time
0,0.124751,0.176218,00:34
1,0.111141,0.159242,00:34
2,0.114572,0.120380,00:35
3,0.123105,0.185561,00:35
4,0.099103,0.140103,00:35
5,0.102806,0.149782,00:35
6,0.095712,0.118776,00:35
7,0.106949,0.098962,00:35
8,0.100995,0.109363,00:35
9,0.103210,0.115479,00:35


Export the Pickled Model.

In [13]:
learn.export('ICECREAMS_V1_1.pkl')